<a href="https://colab.research.google.com/github/Harshitnitw/self-hosted-llm-used-to-retrieve-info-from-website-with-pinecone/blob/main/self_hosted_llm_used_to_retrieve_info_from_website_with_pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip -q install langchain langchain-community langchain-pinecone pinecone-notebooks bitsandbytes accelerate transformers datasets loralib sentencepiece pypdf sentence_transformers unstructured tokenizers xformers pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.6 MB/s eta 0:00:00


In [43]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
import pinecone
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import textwrap
import sys
import os
import torch

In [8]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [9]:
URLs=[
    'https://www.databricks.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models'
]

In [10]:
loaders = UnstructuredURLLoader(urls=URLs)
data = loaders.load()

In [12]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [13]:
text_chunks=text_splitter.split_documents(data)

In [14]:
text_chunks[0]

Document(metadata={'source': 'https://www.databricks.com/blog/mpt-7b'}, page_content='Introducing MPT-7B, the first entry in our MosaicML Foundation Series. MPT-7B is a transformer trained from scratch on 1T tokens of text and code. It is open source, available for commercial use, and matches the quality of LLaMA-7B. MPT-7B was trained on the MosaicML platform in 9.5 days with zero human intervention at a cost of ~$200k.\nLarge language models (LLMs) are changing the world, but for those outside well-resourced industry labs, it can be extremely difficult to train and deploy these models. This has led to a flurry of activity centered on open-source LLMs, such as the LLaMA series from Meta, the Pythia series from EleutherAI, the StableLM series from StabilityAI, and the OpenLLaMA model from Berkeley AI Research.')

In [15]:
text_chunks[1]

Document(metadata={'source': 'https://www.databricks.com/blog/mpt-7b'}, page_content='Today, we at MosaicML are releasing a new model series called MPT (MosaicML Pretrained Transformer) to address the limitations of the above models and finally provide a commercially-usable, open-source model that matches (and - in many ways - surpasses) LLaMA-7B. Now you can train, finetune, and deploy your own private MPT models, either starting from one of our checkpoints or training from scratch. For inspiration, we are also releasing three finetuned models in addition to the base MPT-7B: MPT-7B-Instruct, MPT-7B-Chat, and MPT-7B-StoryWriter-65k+, the last of which uses a context length of 65k tokens!\nOur MPT model series is:\nLicensed for commercial use (unlike LLaMA).\nTrained on a large amount of data (1T tokens like LLaMA vs. 300B for Pythia, 300B for OpenLLaMA, and 800B for StableLM).\nPrepared to handle extremely long inputs thanks to ALiBi (we trained on up to 65k inputs and can handle up to

In [16]:
len(text_chunks)

41

In [17]:
embeddings=HuggingFaceEmbeddings()

<ipython-input-17-5897ca6a0cdb>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings()
<ipython-input-17-5897ca6a0cdb>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings=HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingfac

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [22]:
from pinecone import Pinecone

pc = Pinecone(api_key=API_KEY)
index = pc.Index("llama")

In [33]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [40]:
vector_store.add_texts([t.page_content for t in text_chunks])

['6f1bee63-e0fb-49ba-8f9c-9b801a2befa9',
 '55a4db50-1d07-4ee1-8921-f0f43aee91b1',
 '2c511a30-22be-42d2-a0f2-688795d68cf6',
 'feec5b43-a1c6-46ba-9505-421fd23e003d',
 '0ec99fbd-0b75-4468-9a05-efd5c7a90540',
 'abb011e0-388b-498a-b923-17ff296ae815',
 'b4bfba83-f8ea-4e35-a812-28ce4bd4ec79',
 '9d9a8f8c-e290-4c3d-b54e-7de1bf44917e',
 '15267842-67a3-4dfe-8c00-b16ce11ccc05',
 '680a6b56-b0db-46df-a25f-6be16460b82c',
 '3fee638f-9918-434c-95f4-f74f4700c3c9',
 '9319a531-c35a-4a73-b32b-b78f30c3cc29',
 '84a3f49f-2abd-4a03-8b28-7740a6b0bda0',
 '280872a1-ec7f-48f4-b0b7-6b2d2bd69f38',
 'dd608d44-e87b-4ea5-b551-aeea5a3634e4',
 'e8b3f289-8d36-4970-bd48-d96d3dea63f0',
 '5ffaf71b-4b67-4150-961c-74d0a11dfcd6',
 '5be53042-118a-46c9-9368-2a56dcd98cf7',
 'a7c9b396-f358-4a09-be0b-d9d15d682c49',
 '63247c91-9c27-4704-bdc5-d64ae8829a95',
 'd8b5ffb0-86bd-4b54-86b6-00c902a839d0',
 '9973bf6d-3641-4ef4-bf42-6aca65178041',
 '8dcaaea6-c86f-4014-bcde-bd8f4316b240',
 'ee11b333-0064-49b8-bc7a-cb084597aa60',
 '45570ff7-db66-

In [41]:
model="daryl149/llama-2-7b-chat-hf"

In [45]:
tokenizer=AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [46]:
model=AutoModelForCausalLM.from_pretrained(model,
                                    device_map='auto',
                                    load_in_8bit=True)

config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [47]:
pipe=pipeline("text-generation",
              model=model,
              tokenizer=tokenizer,
              device_map="auto",
              max_new_tokens=512,
              do_sample=True,
              top_k=30,
              num_return_sequences=1,
              model_kwargs={"load_in_8bit": True}
              )

Device set to use cuda:0


In [48]:
llm=HuggingFacePipeline(pipeline=pipe,model_kwargs={'temperature':0})

<ipython-input-48-e4409001cd3a>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm=HuggingFacePipeline(pipeline=pipe,model_kwargs={'temperature':0})


In [49]:
llm.predict("please let me know about you")

<ipython-input-49-f55c8e55b964>:1: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("please let me know about you")


"please let me know about you in brief.\nThe best way to get to know someone is to ask them questions. Here are some questions you can ask your interviewer to learn more about them:\n1. Can you tell me a little bit about your background and how you became interested in this field?\n2. What do you find most rewarding about your work?\n3. How do you handle challenges or difficult situations in the workplace?\n4. Can you tell me about a project or accomplishment that you are particularly proud of?\n5. What are your long-term goals for your career?\n6. What do you think are the most important qualities for someone in this field to have?\n7. Can you tell me about any mentors or role models you have in your field?\n8. How do you stay current with industry trends and developments?\n9. Can you tell me about any significant professional or personal achievements outside of work?\n10. What do you think sets you apart from other people in your field?\nAlways be respectful and professional when ask

In [50]:
from langchain.chains import RetrievalQA

In [51]:
query="How good is Vicuna"

In [52]:
docs=vector_store.similarity_search(query,k=3)
docs

[Document(id='15267842-67a3-4dfe-8c00-b16ce11ccc05', metadata={}, page_content='See Table 1 for a comparison of zero-shot performance between MPT and other models:\nTo ensure apples-to-apples comparisons, we fully re-evaluated each model: the model checkpoint was run through our open source LLM Foundry eval framework with the same (empty) prompt strings and no model-specific prompt tuning. For full details on the evaluation, see the Appendix. In previous benchmarks, our setup is 8x faster than other eval frameworks on a single GPU and seamlessly achieves linear scaling with multiple GPUs. Built-in support for FSDP makes it possible to evaluate large models and use larger batch sizes for further acceleration.\nWe invite the community to use our evaluation suite for their own model evaluations and to submit pull requests with additional datasets and ICL task types so we can ensure the most rigorous possible evaluation.\nMPT-7B-StoryWriter-65k+'),
 Document(id='9d9a8f8c-e290-4c3d-b54e-7de

In [53]:
qa=RetrievalQA.from_chain_type(llm=llm, chain_type="stuff",
                               retriever=vector_store.as_retriever())

In [55]:
result=qa.run(query)
print(result)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

See Table 1 for a comparison of zero-shot performance between MPT and other models:
To ensure apples-to-apples comparisons, we fully re-evaluated each model: the model checkpoint was run through our open source LLM Foundry eval framework with the same (empty) prompt strings and no model-specific prompt tuning. For full details on the evaluation, see the Appendix. In previous benchmarks, our setup is 8x faster than other eval frameworks on a single GPU and seamlessly achieves linear scaling with multiple GPUs. Built-in support for FSDP makes it possible to evaluate large models and use larger batch sizes for further acceleration.
We invite the community to use our evaluation suite for their own model evaluations and to submit pull requests with additional datasets and ICL task types so we can ensure the most rigorous possible

In [63]:
from langchain.prompts import PromptTemplate

custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""Based on the following context, provide a short and direct answer.

Context:
{context}

Question: {question}

Answer (in one or two sentences, without verbose or explanation):"""
)


qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    chain_type_kwargs={"prompt": custom_prompt}
)

In [64]:
result3 = qa.invoke("where did training data for MPT-7B come from?")
print(result3["result"])

Based on the following context, provide a short and direct answer.

Context:
Check out the StreamingDataset blog for more details!
Training Compute
All MPT-7B models were trained on the MosaicML platform with the following tools:
Compute: A100-40GB and A100-80GB GPUs from Oracle Cloud
Orchestration and Fault Tolerance: MCLI and MosaicML platform
Data: OCI Object Storage and StreamingDataset
Training software: Composer, PyTorch FSDP, and LLM Foundry
As shown in Table 3, nearly all of the training budget was spent on the base MPT-7B model, which took ~9.5 days to train on 440xA100-40GB GPUs, and cost ~$200k. The finetuned models took much less compute and were much cheaper – ranging between a few hundred and few thousand dollars each.
Time to Train' is the total runtime from job start to finish, including checkpointing, periodic evaluation, restarts, etc. 'Cost' is computed with pricing of $2/A100-40GB/hr and $2.50/A100-80GB/hr for reserved GPUs on the MosaicML platform.

Efficient Data 